In [ ]:
__path__ = 'models'

LOAD MODEL + TOKENIZER + CONFIG

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

In [4]:
model = load_model(f'{__path__}/model.keras')

with open(f'{__path__}/tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json_string = f.read()
    tokenizer = tokenizer_from_json(tokenizer_json_string)

with open(f'{__path__}/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

In [5]:
max_len = config['max_len']
emotion_labels = config['emotion_labels']
num_classes = config['num_classes']

print(f"\nModel Configuration:")
print(f"  - Max Length: {max_len}")
print(f"  - Number of Classes: {num_classes}")
print(f"  - Vocabulary Size: {config['vocab_size']}")
print(f"  - Emotion Labels: {emotion_labels}")


Model Configuration:
  - Max Length: 40
  - Number of Classes: 6
  - Vocabulary Size: 36540
  - Emotion Labels: ['depressed', 'sad', 'neutral', 'good', 'happy', 'excited']


PREDICT FUNCTIONS

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # Ensure pandas is imported

def predict_emotion(text, show_details=True):
    """
    Predict emotion for a single text

    Args:
        text (str): Input text to classify
        show_details (bool): Whether to show detailed probabilities

    Returns:
        dict: Prediction results
    """

    # preprocess text
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

    # predict
    predictions = model.predict(padded, verbose=0)[0]
    predicted_class = int(np.argmax(predictions))
    confidence = float(predictions[predicted_class])

    # get emotion label (fixed to use list indexing)
    if 0 <= predicted_class < len(emotion_labels):
        emotion = emotion_labels[predicted_class]
    else:
        emotion = f'Class {predicted_class}'


    all_probabilities_dict = {emotion_labels[i]: float(predictions[i])
                             for i in range(len(predictions))}


    result = {
        'text': text,
        'predicted_class': predicted_class,
        'emotion': emotion,
        'confidence': confidence, # Store confidence as float
        'all_probabilities': all_probabilities_dict
    }

    if show_details:
        print(f"\n{'='*60}")
        print(f"Text: '{text}'")
        print(f"{'='*60}")
        print(f"Predicted Emotion: {emotion}")
        print(f"Confidence: {confidence:.2%}")
        print(f"\nAll Probabilities:")
        # Iterate through sorted probabilities for better display
        for emo, prob in sorted(all_probabilities_dict.items(), key=lambda item: item[1], reverse=True):
            bar = '█' * int(prob * 40)
            print(f"  {emo:12s}: {bar:40s} {prob:.2%}")

    return result


def predict_batch(texts):
    """
    Predict emotions for multiple texts

    Args:
        texts (list): List of input texts

    Returns:
        pd.DataFrame: Results with predictions
    """
    results = []

    for text in texts:
        result = predict_emotion(text, show_details=False)
        results.append({
            'text': text[:50] + '...' if len(text) > 50 else text,
            'emotion': result['emotion'],
            'confidence': result['confidence'] # Store confidence as float in the list of dictionaries
        })

    # Create DataFrame and format confidence for display
    df_results = pd.DataFrame(results)
    df_results['confidence'] = df_results['confidence'].apply(lambda x: f"{x:.2%}")

    return df_results


def visualize_prediction(text):
    """
    Visualize prediction probabilities for a single text

    Args:
        text (str): Input text to classify
    """
    result = predict_emotion(text, show_details=False)

    # Prepare data for plotting
    emotions = list(result['all_probabilities'].keys())
    probabilities = list(result['all_probabilities'].values())

    # Create bar plot
    plt.figure(figsize=(12, 6))
    # Adjust colors based on the order in the emotion_labels list
    colors = ['#ff6b6b' if emotions[i] == result['emotion'] else '#4ecdc4'
              for i in range(len(emotions))]

    bars = plt.bar(emotions, probabilities, color=colors, alpha=0.7, edgecolor='black')

    # Highlight predicted class
    # Find the index of the predicted emotion in the current emotions list
    predicted_index = emotions.index(result['emotion'])
    bars[predicted_index].set_linewidth(3)


    plt.title(f'Emotion Prediction Probabilities\nPredicted: {result["emotion"]} ({result["confidence"]:.2%})',
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('Emotion', fontsize=12)
    plt.ylabel('Probability', fontsize=12)
    plt.ylim(0, 1)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for i, (bar, prob) in enumerate(zip(bars, probabilities)):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{prob:.3f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.show()

TESTING

In [19]:
print("\n" + "="*80)
print("EMOTION PREDICTION")
print("="*80)
print("Enter text to predict emotion. Type 'quit' to exit.\n")

# Get user input
text = input("Enter text: ").strip()

if text.lower() in ['quit', 'exit', 'q']:
    print("Exiting interactive mode...")

if not text:
    print("Please enter some text.\n")

# Predict and show results
predict_emotion(text, show_details=True)

# Ask if user wants to visualize
viz = input("\nVisualize this prediction? (y/n): ").strip().lower()
if viz == 'y':
    visualize_prediction(text)
print()


EMOTION PREDICTION
Enter text to predict emotion. Type 'quit' to exit.

Enter text: im so happy today

Text: 'im so happy today'
Predicted Emotion: sad
Confidence: 82.98%

All Probabilities:
  sad         : █████████████████████████████████        82.98%
  good        : ██                                       6.63%
  excited     : █                                        4.31%
  depressed   :                                          2.47%
  happy       :                                          2.06%
  neutral     :                                          1.55%

Visualize this prediction? (y/n): n

